### Аргументы; RVO / NRVO / copy elision

<br />

##### Calling convention

__Вопрос-наброс:__ Как компилятор должен сформировать вызов функции?

```c++
int my_max(int a, int b)
{
    return a < b ? b : a;
}
```

- Как передать параметры - через регистры (а если их не хватит?), через стек, через heap?
- Как их расположить, слева направо или справа налево?
- Кто ответственный за переключение регистра, отвечающего за адрес выполняемой инструкции?
- Должна ли вызываемая функция восстановить состояние регистров как было до вызова?
- ...

<br />

Соглашение о вызове — спецификация вызова функций:
* способы передачи параметров
* способы передачи управления
* способы передачи результата
* способы возврата управления

__Примеры:__

https://docs.microsoft.com/en-us/cpp/cpp/argument-passing-and-naming-conventions?view=vs-2019

* `cdecl` - для языка С для функций с переменным числом аргументов
    * аргументы передаются через стек справа налево
    * вызывающий чистит стэк (он знает число аргументов)
    * функция обязана восстановить состояние регистров как было до вызова
        * компилятору на каждую функцию нужно генерировать код по сохранению регистров (prolog) и восстановлению (epilog)
* `stdcall`
    * аргументы передаются через стек справа налево
    * функция сама чистит стек
* `fastcall`
    * параметры передаются через регистры (если влезают в них) - самый быстрый способ передачи
    * функция сама чистит стек
    * соглашение используется для вызова процедур и функций, не экспортируемых из исполняемого модуля и не импортируемых извне
        * _А ты не забыл добавить `static` для функции, которая не нужна наружу?_
* `thiscall`
    * соглашение для вызова методов класса

__Замечание:__ К вопросу о том что даёт оптимизация inlining по части вызова функции:
* убирает дополнительные расходы на вызов функции
* даёт гарантии о состоянии регистров
    * если происходит вызов функции - часть регистров может потерять информацию
    * если вызова нет, компилятор имеет гарантии по значениям в регистрах и может их переиспользовать, уменьшив доступы в память
    
    ```c++
    void f()
    {
        int x = std::rand();
        std::cout << x; // x должен появиться в регистрах, чтобы быть использованным
        
        do_some_work();
        
        int y = x + 1; // если нет гарантий на сохранение регистров,
                       // возможно, придётся перечитывать x со стека заново
    }
    ```

<br />

##### Передача аргументов в функцию

[Объяснение на cppcoreguidelines](https://github.com/isocpp/CppCoreGuidelines/blob/master/CppCoreGuidelines.md#f15-prefer-simple-and-conventional-ways-of-passing-information)

Правила для обычных смертных (до С++11)

![](param-passing-normal.png)

Правила для тех, кто умеет в move-семантику (после С++11)

![](param-passing-advanced.png)

Ещё есть perfect forwarding и universal references, к ним вернёмся в продолжении курса [[1](https://eli.thegreenplace.net/2014/perfect-forwarding-and-universal-references-in-c)][[2](https://en.cppreference.com/w/cpp/utility/forward)].

__Упражнение:__ Как организовать аргументы и возвращаемые значения для функций?

<br />
<details>
<summary>Посчитать сумму <tt>float</tt>-ов в <tt>std::vector</tt>.</summary>
<p>

```c++
float sum(const std::vector<float>& v);
```
</p>
</details>

<details>
<summary>По <tt>int n</tt> посчитать <tt>std::vector</tt> первых <tt>n</tt> чисел ряда Фибоначи</summary>
<p>

```c++
std::vector<int> fib(int n);
```

</p>
</details>

<details>
<summary>Создать <tt>std::unique_ptr&lt;Cat&gt;</tt> по имени <tt>std::string</tt></summary>
<p>

```c++
std::unique_ptr<Cat> create_cat(const std::string& name);
std::unique_ptr<Cat> create_cat(std::string name);  // + std::move
std::unique_ptr<Cat> create_cat(std::string&& name);  // + std::move
```

</p>
</details>

<details>
<summary>Отсортировать <tt>std::vector&lt;int&gt;</tt></summary>
<p>

```c++
void sort_inplace(std::vector<int>& v);
```

</p>
</details>

<details>
<summary>Отсортировать <tt>std::array&lt;int, 1024&gt;</tt></summary>
<p>

```c++
void sort_inplace(std::array<int, 1024>& v);
```

В чём разница с `std::vector`?

</p>
</details>

<details>
<summary>Найти максимальный элемент в <tt>std::vector&lt;int&gt;</tt></summary>
<p>

```c++
const int* max_integer(const std::vector<int>& v);
std::vector<int>::const_iterator max_integer(const std::vector<int>& v);
```

</p>
</details>

<details>
<summary>Найти площадь треугольника по формуле Герона</summary>
<p>

```c++
std::optional<double> get_triangle_area(double a, double b, double c);
double get_triangle_area(double a, double b, double c); // + throw exception

// optional usage:
std::optional<double> maybeArea = get_triangle_area(3, 4, 5);
if (maybeArea.has_value())
    std::cout << "area is: " << maybeArea.value() << std::endl;

// отступление: более корректным был бы такой интерфейс:
// double get_area(const Triangle& triangle);
//
// в этом варианте, если объект типа Triangle существует,
// значит он корректен, и внутри всех функций, работающих
// с треугольником
```

</p>
</details>

<br />

##### Возвращаемое значение, RVO/NRVO

https://www.fluentcpp.com/2016/11/28/return-value-optimizations/

Рассмотрим пример:

```c++
House build_house() {
    House house;
    house.add_floor(make_floor());
    house.add_walls(make_walls());
    return house;
}
```

```c++
House h = build_house();
```

В общем случае нужно сделать копию из локальной переменной `house` в возвращаемое значение.

Но компилятор _иногда может_ копию оптимизировать.

RVO:

```c++
House build_house() {
    // можно сконструировать |House| прямо в месте,
    // отведённом под результат без промежуточной копии
    return House(make_floor(), make_walls());
}
```

```c++
House build_house(bool stone) {
    if (stone)
        return House(make_stone_floor(), make_stone_walls());
    else
        return House(make_brick_floor(), make_brick_walls());
}
```

NRVO:

```c++
House build_house() {
    // можно сконструировать |house| прямо в месте,
    // отведённом под результат, без промежуточной копии,
    // изменения можно проводить прямо в объекте возврата
    House house;
    house.add_floor(make_floor());
    house.add_walls(make_walls());
    return house;
}
```

```c++
House build_house(bool stone) {
    House house;
    house.add_floor(stone ? make_stone_floor() : make_brick_floor());
    house.add_walls(stone ? make_stone_walls() : make_brick_walls());
    return house;
}
```

<br />

Когда оптимизации не работают:

```c++
House build_house(int id) {
    House stone_house;
    stone_house.add_floor(make_stone_floor());
    stone_house.add_walls(make_stone_walls());
    
    House brick_house;
    brick_house.add_floor(make_brick_floor());
    brick_house.add_walls(make_brick_walls());
   
    // компилятор не знает из какого объекта
    // будет возвращаться значение, поэтому должен
    // построить оба и скопировать нужный в результат.
    return is_stone_house(id) ? stone_house : brick_house;
}
```

```c++
House makeup_house(House house)
{
    house.cleanup_floor();
    house.change_walls();
    
    // объект |house| уже сконструирован,
    // придётся его либо скопировать в результат
    // либо сделать move
    return house;
}
```

<br />

**Рекомендации до С++17**:
* либо всегда конструировать объект в `return`

    ```c++
    House build() {
        ...
        return House(...);
    }
    ```

* либо один объект для возврата в функции

    ```c++
    House build(bool stone) {
        House house;
        if (stone) {
            ... // setup stone house
        } else {
            ... // setup another house
        }        
        return house;
    }    
    ```

<br />

##### Copy elision

Оптимизации RVO/NRVO получили развитие в [copy elision](https://en.cppreference.com/w/cpp/language/copy_elision) в С++17.

"Copy elision does not elide copies"

Чтобы избавиться от лишних копий / перемещений временных объектов стандартизаторы языка пошли дальше: вместо разрешения оптимизаций были докручены правила работы компилятора с prvalue - выражениями: prvalue можно назвать "нематериализованным объектом", и его конструирование откладывается до момента появления итогового объекта. 

Note: the rule above does not specify an optimization: C++17 core language specification of prvalues and temporaries is fundamentally different from that of the earlier C++ revisions: there is no longer a temporary to copy/move from. Another way to describe C++17 mechanics is "unmaterialized value passing": prvalues are returned and used without ever materializing a temporary.

Пример 1:
    
```c++
T f() {
    return T();
}

f();  // only one call to default constructor of T
```

Пример 2:
    
```c++
T f() {
    return T();
}

T x = T(T(f())); // only one call to default constructor of T, to initialize x
```

<br />

**Рекомендации после С++17:**

* предпочтительнее конструировать объект в return, чтобы отработал copy elision, компилятор его гарантирует

    ```c++
    House build() {
        ...
        return House(...);
    }
    ```

* если без именованного объекта не обойтись, желательно заиспользовать NRVO (и молиться на добрую волю компилятора)

    ```c++
    House build(bool stone) {
        House house;
        if (stone) {
            ... // setup stone house
        } else {
            ... // setup another house
        }
        return house;
    }
    ```

<br />

__Для понимания__:
* [CppCon 2018: Arthur O'Dwyer “Return Value Optimization: Harder Than It Looks”. Доклад про RVO и внутреннюю механику](https://www.youtube.com/watch?v=hA1WNtNyNbo)
* [copy elision does not elide copies](https://devblogs.microsoft.com/cppblog/guaranteed-copy-elision-does-not-elide-copies/) 